### Workflow
1. Extract the start and end vertices from each path feature in the LCP pipeline network --> Nodes
1. Dissolve these to eliminate overlappiung duplicates.
1. Add a unique value to each node
1. Split the path features by these nodes; these will become our edges
1. Spatially join the node feature IDs to the 

In [1]:
#Imports
import arcpy
import numpy as np
import pandas as pd

In [2]:
#Set arcpy environments
arcpy.env.workspace = '..\\scratch\\scratch.gdb'
arcpy.env.scratchWorkspace = '..\\scratch\\scratch.gdb'
arcpy.env.overwriteOutput = True

In [3]:
#Set inputs
lcp_fc = '..\\scratch\\LCPNetwork.shp'                          # LCP feature class
bgprj_fc = '..\\OptimizeViewer\\OptimizeViewer.gdb\\Biogas_prj' # Biogas source locations, w/attributes

In [4]:
#Convert LCP vertices to points
lcp_ends = arcpy.management.FeatureVerticesToPoints(in_features=lcp_fc,
                                                    out_feature_class='memory/LCP_vertices',
                                                    point_location='BOTH_ENDS').getOutput(0)

In [5]:
#Dissolve the points
lcp_nodes = arcpy.management.Dissolve(in_features=lcp_ends, 
                                      out_feature_class="memory/LCP_nodes", 
                                      multi_part="SINGLE_PART").getOutput(0)

In [6]:
#Add a unique ID field to the lcp_split
arcpy.management.AddField(in_table=lcp_nodes,field_name='Node_ID',field_type='SHORT')
#Compute the unique ID field as the FID
arcpy.management.CalculateField(in_table=lcp_nodes, field='Node_ID',
                                expression='!OBJECTID!',expression_type='PYTHON3')

<Result 'memory\\LCP_nodes'>

In [12]:
#Split the LCP at the vertices
lcp_split = arcpy.management.SplitLineAtPoint(in_features=lcp_fc,
                                              point_features=lcp_nodes,
                                              out_feature_class='LCP_split_all',
                                              search_radius='10 meters').getOutput(0)

#Remove features with no length
lcp_split = arcpy.analysis.Select(in_features=lcp_split,
                                  out_feature_class='memory/LCP_split_keep',
                                  where_clause="Shape_Length > 0"
                                 ).getOutput(0)

#Remove the temp dataset
arcpy.management.Delete('LCP_split_all')

<Result 'true'>

In [13]:
#Spatially join the nodes to the lines
if (arcpy.Exists('lcp_edges')): arcpy.management.Delete('lcp_edges')
lcp_edges = arcpy.analysis.SpatialJoin(target_features=lcp_split,
                                       join_features=lcp_nodes,
                                       out_feature_class='lcp_edges',
                                       join_operation='JOIN_ONE_TO_MANY',
                                       match_option='WITHIN_A_DISTANCE',
                                       search_radius='50 meters'
                                      ).getOutput(0)

The `lcp_edges` feature class now includes two features for each segment in the LCP network. The two segments share the same `TARGET_FID` value, and the `PtID` value corresponds to the node on either side. So we can construct an edge list by pivoting the attribute table: row  = `TARGET_FID` and column = `PtID`...

In [14]:
#Convert the table to a numpy array
arr_edges = arcpy.da.TableToNumPyArray(in_table=lcp_edges,field_names=['TARGET_FID','Node_ID'])

#And to a Pandas dataframe
df_edges = pd.DataFrame(arr_edges)

In [38]:
#Ensure there are no more than two nodes for each edge
df_edges_count = df_edges.groupby('TARGET_FID').count()
if ((df_edges_count.Node_ID > 2).sum() == 0 & (df_edges_count.Node_ID > 2).sum() == 0) :
    print("all good!")
else: print ("Check values!")

all good!


In [39]:
#Get start node values to a column
df_edge1 = df_edges.groupby('TARGET_FID').agg('first')
df_edge2 = df_edges.groupby('TARGET_FID').agg('last')
df_edgelist = df_edge1.join(df_edge2,lsuffix='From',rsuffix='To')

In [ ]:
#Join the 
arr_nodes = arcpy.da.TableToNumPyArray(in_table=lcp_ends,field_names['Total_Pote','ORIG_FID'])

In [69]:
#Spatially join the Node IDs to the BG points
bg_sites = arcpy.analysis.SpatialJoin(target_features=bgprj_fc,
                           join_features=lcp_nodes,
                           out_feature_class='memory/BG_sites',
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_COMMON", 
                           match_option="WITHIN_A_DISTANCE", 
                           search_radius="100 Meters").getOutput(0)

In [74]:
#Convert to Pnadas dataframe (via Numpy array)
dfSites = pd.DataFrame(arcpy.da.TableToNumPyArray(in_table=bg_sites,
                                                  field_names=['Node_ID','Total_Potential_Methane_Yield_1']))
dfSites.head()

,Node_ID,Total_Potential_Methane_Yield_1
0,576,357.493608
1,555,421.200530
2,158,309.706272
3,605,399.176973
4,507,443.912323


In [81]:
#Join methane yield to edge list
df_edgelist2 = pd.merge(left=df_edgelist,left_on='Node_IDFrom',right=dfSites,right_on='Node_ID',how='left')
df_edgelist2.fillna(0,inplace=True)
df_edgelist2.drop('Node_ID',axis=1,inplace=True)
df_edgelist2.columns = ['From_node','To_node','Yield_scf_y']
df_edgelist2.head()

,From_node,To_node,Yield_scf_y
0,33,34,0.0
1,25,34,0.0
2,5,25,0.0
3,1,5,0.0
4,216,245,0.0


In [82]:
#Write the edge list to a csv
df_edgelist2.to_csv('../scratch/EdgeList.csv',index=False)